In [1]:
# MFCC
# Import modules
import numpy as np
import scipy.io.wavfile
from scipy.fftpack import dct
# Import data
fs, data = scipy.io.wavfile.read('digits/1a.wav')
# Set window size
win_size = 0.025 # ms
win_shift = 0.010 # ms
# Convert from seconds to samples
win_len, win_step = win_size*fs, win_shift*fs
nsamples = len(data)
win_len = int(round(win_len))
win_step = int(round(win_step))
# Round up for number of windows so can't be 0
nwins = int(np.ceil(float(np.abs(nsamples - win_len))/win_step))
# Pad signal so can divide exactly into windows
npad = nwins * win_step + win_len
data = np.append(data, np.zeros((npad - nsamples))) # Pad signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
# Extract windows
indices = np.tile(np.arange(0, win_len), (nwins, 1)) + np.tile(np.arange(0, nwins*win_step, win_step), (win_len, 1)).T
wins = data[indices.astype(np.int32, copy = False)]
# Apply hamming window
wins *= np.hamming(win_len)
# Calculate fft
NFFT = 512
# Extract magnitude of the FFT over windows
mag_wins = np.absolute(np.fft.rfft(wins, NFFT))
# Convert to power Spectrum
pow_wins = ((1.0/NFFT)*((mag_wins)**2))
# Apply Mel-frequency triangular filters
# Number of filters
nfilts = 40
# Mel-frequency range
low_freq_mel = 0
high_freq_mel = (2595 * np.log10(1 + (fs/2)/700))
# Create equally spaced points in Mel scale
mel_pts = np.linspace(low_freq_mel, high_freq_mel, nfilts + 2)
# Calculate Hz correspondence
hz_pts = (700*(10**(mel_pts/2595) - 1))
bin = np.floor((NFFT + 1)*hz_pts/fs)
# Calculate Mel-filter banks
fbank = np.zeros((nfilts, int(np.floor(NFFT/2 + 1))))
for m in range(1, nfilts + 1):
    # Calculate edge and center frequencies
    f_m_minus = int(bin[m - 1])
    f_m = int(bin[m])
    f_m_plus = int(bin[m + 1])
    for k in range(f_m_minus, f_m):
        fbank[m - 1, k] = (k - bin[m - 1])/(bin[m] - bin[m - 1])
    for k in range(f_m, f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k)/(bin[m + 1] - bin[m])
fbanks = np.dot(pow_wins, fbank.T)
fbanks = np.where(fbanks == 0, np.finfo(float).eps, fbanks)  # Numerical Stability
# Convert to dB
fbanks = 20 * np.log10(fbanks)
# Extract Mel-frequency cepstral coefficients (MFCCs)
nceps = 12
mfccs = dct(fbanks, type = 2, axis = 1, norm = 'ortho')[:, 1:(nceps + 1)] # Keep 2nd onwards

In [2]:
# DTW
# Import modules
import math
import numpy as np
import operator
# Create function to calculate Euclidean distance between feature vectors
def eudistVec(x, y):
    # Convert scalar inputs to vectors
    if not type(x) is list:
        x = [x]
    if not type(y) is list:
        y = [y]
        # Return Euclidean distance
    return math.sqrt(sum(list(map(lambda x, y: (x - y)**2, x, y))))
# Create function to do dynamic time warp
def dynamTimeWarp(x, y):
    # Convert scalar inputs to vectors
    if not type(x) is list:
        x = x.tolist()
    if not type(y) is list:
        y = y.tolist()
    # Initialize path matrix
    DTW = np.zeros([len(x), len(y)])
    # Calculate paths
    for i in range(len(x)):
        for j in range(len(y)):
            dist = eudistVec(x[i], y[j])
            if not i and not j:
                choices = float('inf'), float('inf'), 0
            elif not i and j:
                choices = float('inf'), DTW[i, j-1], float('inf')
            elif i and not j:
                choices = DTW[i-1, j], float('inf'), float('inf')
            else:
                choices = DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1]
            DTW[i, j] = dist + min(choices)
    # Return score/error of best path (lower is better)
    return DTW[-1, -1]
# Test algorithm
A = [[0, 0], [1, 0], [2, 0], [3, 0], [0, 0]]
B = [[0, 0], [0, 0], [1, 1.5], [2, 1], [3, 0.5], [0, 0], [0, 0], [0, 0]]
score = dynamTimeWarp(A, B)
score

3.0

In [3]:
# Test algorithm on MFCCs
dynamTimeWarp(mfccs, mfccs)

0.0